# **Membuat Aplikasi Chat bot**

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
from random import choice
import os
from keras.models import load_model

In [21]:
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Step 1: Download all the data from github

In [9]:
!wget https://github.com/yannaakbar/RNN-seq2seq-chatbot-mental-illness/archive/refs/heads/main.zip -O chatbot_nlp.zip

--2022-12-11 14:03:56--  https://github.com/yannaakbar/RNN-seq2seq-chatbot-mental-illness/archive/refs/heads/main.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/yannaakbar/RNN-seq2seq-chatbot-mental-illness/zip/refs/heads/main [following]
--2022-12-11 14:03:56--  https://codeload.github.com/yannaakbar/RNN-seq2seq-chatbot-mental-illness/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘chatbot_nlp.zip’

chatbot_nlp.zip         [ <=>                ] 861.95K  --.-KB/s    in 0.09s   

2022-12-11 14:03:56 (9.24 MB/s) - ‘chatbot_nlp.zip’ saved [882634]



In [10]:
!unzip chatbot_nlp.zip

Archive:  chatbot_nlp.zip
43b42698b912fa01d4c6460de96eb91b9fd3ac3e
   creating: RNN-seq2seq-chatbot-mental-illness-main/
  inflating: RNN-seq2seq-chatbot-mental-illness-main/.gitignore  
   creating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/
   creating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/dataset/
  inflating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/dataset/intents.json  
   creating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/
  inflating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/classes.pkl  
  inflating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/rnn_model.h5  
  inflating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/words.pkl  
  inflating: RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model_plot.png  
  inflating: RNN-seq2seq-chatbot-mental-illness-main/README.md  


# Step 2: Load Model

In [12]:
dir_path = '/content/RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model'
files_list = os.listdir(dir_path + os.sep)

In [14]:
#Load model 
#memanggil model yang sudah dibuat
model = load_model('/content/RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/rnn_model.h5')
intents = json.loads(open('/content/RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/dataset/intents.json', encoding='utf-8').read())
words = pickle.load(open('/content/RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/words.pkl','rb'))
classes = pickle.load(open('/content/RNN-seq2seq-chatbot-mental-illness-main/Chatbot-RNN-Seq2Seq/model/classes.pkl','rb'))

# Step 3: Membuat Fungsi Clean up sentence 

In [15]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Step 4: Membuat Fungsi Bag of Words

In [16]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

# Step 5: Membuat Fungsi Predict Class

In [17]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Step 6: Membuat Fungsi untuk mengambil atau memberikan responses

In [19]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = choice(i['responses'])
            break
        else:
            result = "You must ask the right questions"
    return result

# Step 2: Membuat Fungsi untuk memberikan respons dan menjalankan chatbot

In [22]:
# FUngsi untuk menjalankan aplikasi
def response(text):
    ints=predict_class(text,model)
    response=getResponse(ints,intents)
    return response

while(1):
    x=input()
    print(response(x))
    # tts = gTTS(response(x), lang='id')
    # tts.save('/content/bot.wav')
    # time.sleep(0.08)
    # ipd.display(ipd.Audio('/content/bot.wav', autoplay=True))
    print("="*60 + "\n")
    if x.lower() in ['bye','goodbye','get lost','see you']:  
        break

hai
1/1 [==============================] - 0s 252ms/step
Helo! Ada masalah mental health ya?

pantun
1/1 [==============================] - 0s 19ms/step
Anak elang mulailah terbang, Ambilkan dedak berilah makan, Janganlah abang merasa bimbang, Segala kehendak aku tunaikan.

mentaL Ilness
1/1 [==============================] - 0s 17ms/step
Mental illness, disebut juga dengan gangguan mental atau jiwa, adalah kondisi kesehatan yang memengaruhi pemikiran, perasaan, perilaku, suasana hati, atau kombinasi diantaranya. Kondisi ini dapat terjadi sesekali atau berlangsung dalam waktu yang lama (kronis)

mental illnessss
1/1 [==============================] - 0s 18ms/step
Mental illness, disebut juga dengan gangguan mental atau jiwa, adalah kondisi kesehatan yang memengaruhi pemikiran, perasaan, perilaku, suasana hati, atau kombinasi diantaranya. Kondisi ini dapat terjadi sesekali atau berlangsung dalam waktu yang lama (kronis)

kesehatan mental
1/1 [==============================] - 0s 17ms/st